# 📘 Chapter 4 — Training Models

---

## A) Where to Read (Pinpoint References)

From `4_Training Models.pdf`:

| Topic | Keywords to Find |
|---|---|
| Linear Regression | `Linear Regression`, `MSE`, `cost function`, `theta` |
| The Normal Equation | `Normal Equation`, `X^T X`, `inverse`, `pseudoinverse` |
| Gradient Descent *(next)* | `Gradient Descent`, `learning rate`, `Batch Gradient Descent` |

---

## B) Explanation

### 1) What Linear Regression Is Trying to Do

We have training data:
* **Inputs (features):** $\mathbf{x}$
* **Targets (labels):** $y$

**Goal:** learn a prediction rule that maps inputs to outputs.

For one feature:

$$\hat{y} = \theta_0 + \theta_1 x$$

* $\theta_0$ = bias / intercept
* $\theta_1$ = slope / weight

For many features, add a constant $x_0 = 1$ so the bias fits naturally:

$$\mathbf{x} = [1, x_1, x_2, \dots, x_n]$$

$$\hat{y} = \mathbf{x}^\top \boldsymbol{\theta}$$

> Same idea: **"weighted sum of features + bias"**

---

### 2) How We Decide What "Best Parameters" Means (The Cost Function)

We need a score that measures **"how wrong"** our predictions are.

The chapter uses **Mean Squared Error (MSE)**:

$$J(\boldsymbol{\theta}) = \frac{1}{m}\sum_{i=1}^{m}(\hat{y}^{(i)} - y^{(i)})^2$$

* $m$ = number of training examples
* $(\hat{y}^{(i)} - y^{(i)})$ = error (residual)
* Squaring makes errors positive and **punishes large errors more**

> 📝 **Key idea:** Training linear regression = choose $\boldsymbol{\theta}$ that **minimizes MSE**.

---

### 3) The Vector/Matrix View (Why the Book Likes It)

Stack all training examples into a matrix:

* $\mathbf{X}$ = design matrix (each row is one instance, including leading `1` for bias)
* $\mathbf{y}$ = vector of targets
* $\boldsymbol{\theta}$ = parameter vector

Predictions for **all instances at once**:

$$\hat{\mathbf{y}} = \mathbf{X}\boldsymbol{\theta}$$

> This makes formulas **compact** and computation **efficient**.

---

### 4) Normal Equation (Direct Solution — No Iteration)

Instead of "try values until it's good", linear regression has a **direct minimizer**:

$$\hat{\boldsymbol{\theta}} = (\mathbf{X}^\top \mathbf{X})^{-1}\mathbf{X}^\top \mathbf{y}$$

**When it's nice:**
* Simple, one-shot solution
* Great when number of features isn't huge

**Practical note ⚠️:**
* Sometimes $(\mathbf{X}^\top \mathbf{X})$ isn't invertible (or is numerically unstable)
* In practice we use a numerically stable method (pseudo-inverse / SVD) under the hood

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

np.random.seed(42)

# Synthetic data: y = 4 + 3x + noise
m = 100
X = 2 * np.random.rand(m, 1)
y = 4 + 3 * X + np.random.randn(m, 1)

# Normal Equation
X_b = np.c_[np.ones((m, 1)), X]          # add x0 = 1 (bias feature)
theta_best = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y

print("theta_best (Normal Equation):", theta_best.ravel())

# scikit-learn check
lin_reg = LinearRegression()
lin_reg.fit(X, y)
print("sklearn intercept_, coef_:", lin_reg.intercept_[0], lin_reg.coef_[0, 0])

theta_best (Normal Equation): [4.21509616 2.77011339]
sklearn intercept_, coef_: 4.215096157546747 2.7701133864384837


# 📘 Chapter 4 — Topic 2: Gradient Descent for Linear Regression (Batch GD)

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, find subsection **"Gradient Descent"** → **"Batch Gradient Descent"**

Keywords: `Gradient Descent`, `Batch Gradient Descent`, `learning rate`, `eta`, `partial derivatives`

---

## B) Explanation

### 1) Why We Need Gradient Descent

The Normal Equation is a direct solution, but it becomes **costly when the number of features is large**.

> **Gradient Descent idea:** instead of solving in one shot, start with some $\boldsymbol{\theta}$ and repeatedly improve it by taking small steps **downhill** on the cost function.

---

### 2) The "Downhill" Direction = The Gradient

For linear regression, the cost function is MSE:

$$J(\boldsymbol{\theta}) = \frac{1}{m}\sum_{i=1}^{m}(\hat{y}^{(i)} - y^{(i)})^2$$

The gradient tells us which direction **increases** the cost the most. To **decrease** the cost, we move in the **opposite direction**:

$$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \eta \nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$$

* $\eta$ = **learning rate** (step size)

---

### 3) Batch Gradient Descent

Batch GD uses the **entire training set** to compute the gradient at each step.

In matrix form (with $X_b$ including the column of 1s):

$$\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta}) = \frac{2}{m}X_b^T(X_b\boldsymbol{\theta} - \mathbf{y})$$

So the update becomes:

$$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \eta\frac{2}{m}X_b^T(X_b\boldsymbol{\theta} - \mathbf{y})$$

---

### 4) Learning Rate Intuition

| Learning Rate | Effect |
|---|---|
| Too small | Training is **slow** — many steps to converge |
| Too large | **Overshoot** the minimum — may diverge |
| Just right | Smooth convergence to minimum |

> 📝 **Key idea:** Gradient Descent is not a one-shot formula. It is an **iterative improvement process** guided by the slope of the cost function.

In [2]:
import numpy as np

np.random.seed(42)

# Data: y = 4 + 3x + noise
m = 100
X = 2 * np.random.rand(m, 1)
y = 4 + 3 * X + np.random.randn(m, 1)

X_b = np.c_[np.ones((m, 1)), X]  # add bias feature

eta = 0.1
n_iterations = 1000

theta = np.random.randn(2, 1)  # random init

for iteration in range(n_iterations):
    gradients = (2/m) * X_b.T @ (X_b @ theta - y)
    theta = theta - eta * gradients

print("theta (Batch GD):", theta.ravel())

theta (Batch GD): [4.21509616 2.77011339]


# 📘 Chapter 4 — Topic 3: Stochastic Gradient Descent (SGD)

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, find subsections:
* **"Stochastic Gradient Descent"**
* **"Mini-batch Gradient Descent"**
* **"Learning Schedules"**

Keywords: `Stochastic`, `Mini-batch`, `learning schedule`, `simulated annealing`, `t0`, `t1`

---

## B) Explanation

### 1) What Changes vs Batch Gradient Descent?

* **Batch GD:** uses all $m$ instances to compute the gradient each step
* **SGD:** uses just **1 instance** (picked randomly) per step

So instead of:

$$\nabla J(\boldsymbol{\theta}) = \frac{2}{m}X_b^T(X_b\boldsymbol{\theta} - \mathbf{y})$$

SGD approximates the gradient using a single training example $(\mathbf{x}^{(i)}, y^{(i)})$:

$$\nabla J(\boldsymbol{\theta}) \approx 2\,\mathbf{x}^{(i)}(\mathbf{x}^{(i)\top}\boldsymbol{\theta} - y^{(i)})$$

---

### 2) Why SGD Is Useful

| | Batch GD | SGD |
|---|---|---|
| Gradient quality | Exact | Noisy approximation |
| Cost per step | Expensive ($m$ samples) | Cheap (1 sample) |
| Scalability | Poor on huge datasets | Scales well |
| Convergence path | Smooth | Jiggles around minimum |

> 📝 **Key idea:** SGD trades gradient accuracy for speed. It often reaches a "pretty good" solution much faster than Batch GD.

---

### 3) The Learning Schedule (Very Important)

Because SGD is noisy, we **decrease the learning rate over time**:
* **Big steps early** → fast progress
* **Smaller steps later** → stabilize around the minimum

> The chapter relates this to **"simulated annealing"**: gradually reducing randomness so the system settles into a minimum.

A common simple schedule:

$$\eta(t) = \frac{t_0}{t + t_1}$$

* $t$ grows with iterations
* $\eta(t)$ slowly decreases over time
* $t_0$ and $t_1$ are schedule hyperparameters you tune

---

### 4) Mini-Batch Gradient Descent (The Middle Ground)

| Variant | Instances per step |
|---|---|
| Batch GD | All $m$ |
| SGD | 1 |
| **Mini-batch GD** | Small batch (e.g. 32, 64, 128) |

Mini-batch GD is the **most used in practice** — it gets:
* More stable gradients than SGD
* Much faster updates than full Batch GD
* GPU efficiency (hardware loves batches)

> 📝 **Key idea:** When people say "SGD" in deep learning, they almost always mean **mini-batch SGD**.

In [3]:
import numpy as np

np.random.seed(42)

# Data: y = 4 + 3x + noise
m = 100
X = 2 * np.random.rand(m, 1)
y = 4 + 3 * X + np.random.randn(m, 1)
X_b = np.c_[np.ones((m, 1)), X]

# Learning schedule: eta(t) = t0 / (t + t1)
t0, t1 = 5, 50
def learning_rate(t):
    return t0 / (t + t1)

n_epochs = 50
theta = np.random.randn(2, 1)

t = 0
for epoch in range(n_epochs):
    # shuffle indices each epoch
    for i in np.random.permutation(m):
        xi = X_b[i:i+1]
        yi = y[i:i+1]
        gradients = 2 * xi.T @ (xi @ theta - yi)
        eta = learning_rate(t)
        theta = theta - eta * gradients
        t += 1

print("theta (SGD):", theta.ravel())

theta (SGD): [4.21443559 2.76905559]


# 📘 Chapter 4 — Topic 4: Mini-Batch Gradient Descent

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, find subsection **"Mini-Batch Gradient Descent"**

Keywords: `Mini-batch`, `batch size`, `Stochastic`, `Batch`, `GPU`

---

## B) Explanation

### 1) Definition

Mini-batch GD is the **"middle ground"** between Batch GD and SGD:

| Variant | Instances per step |
|---|---|
| Batch GD | All $m$ instances |
| SGD | 1 instance |
| **Mini-batch GD** | Small batch of size $b$ (e.g. 16, 32, 64, 128) |

Each update uses only $b$ examples:

$$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \eta \cdot \nabla_{\boldsymbol{\theta}}J_{batch}(\boldsymbol{\theta})$$

---

### 2) Why It's Popular in Practice

**Compared to SGD:**
* Gradient is **less noisy** → updates are steadier
* Converges more smoothly (less random bouncing)

**Compared to Batch GD:**
* Each step is **cheaper** than using all $m$
* Works very well with **vectorized operations** (GPUs/CPUs love matrix batches)

---

### 3) Key Tradeoff: Batch Size $b$

| Batch Size | Behavior |
|---|---|
| Small $b$ | Faster steps, noisier updates → more like SGD |
| Large $b$ | Smoother, more expensive steps → more like Batch GD |

> 📝 **Key idea:** Mini-batch GD is the **default choice in modern deep learning**. When people say "SGD" in practice, they almost always mean mini-batch SGD with a learning schedule.

---

### 4) Full Comparison — All Three Variants

| | Batch GD | SGD | Mini-batch GD |
|---|---|---|---|
| Gradient quality | Exact | Very noisy | Moderately noisy |
| Cost per step | High | Very low | Low |
| Convergence path | Smooth | Erratic | Steady |
| GPU efficiency | Poor | Poor | **Excellent** |
| Used in practice | Rarely | Sometimes | **Most common** |

In [4]:
import numpy as np

np.random.seed(42)

# Data: y = 4 + 3x + noise
m = 100
X = 2 * np.random.rand(m, 1)
y = 4 + 3 * X + np.random.randn(m, 1)
X_b = np.c_[np.ones((m, 1)), X]

# Learning schedule (same idea as SGD)
t0, t1 = 5, 50
def learning_rate(t):
    return t0 / (t + t1)

n_epochs = 200
batch_size = 20

theta = np.random.randn(2, 1)
t = 0

for epoch in range(n_epochs):
    shuffled_idx = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_idx]
    y_shuffled = y[shuffled_idx]

    for start in range(0, m, batch_size):
        end = start + batch_size
        X_batch = X_b_shuffled[start:end]
        y_batch = y_shuffled[start:end]

        gradients = (2/len(X_batch)) * X_batch.T @ (X_batch @ theta - y_batch)
        eta = learning_rate(t)
        theta = theta - eta * gradients
        t += 1

print("theta (Mini-batch GD):", theta.ravel())

theta (Mini-batch GD): [4.20609049 2.77894893]


# 📘 Chapter 4 — Topic 5: Polynomial Regression

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, find section **"Polynomial Regression"** (comes after the Gradient Descent family)

Keywords: `Polynomial Regression`, `PolynomialFeatures`, `overfitting`, `degree`, `learning curves`

---

## B) Explanation

### 1) Motivation: Linear Models Can Be "Too Straight"

Plain linear regression fits:

$$\hat{y} = \theta_0 + \theta_1 x$$

That's a **straight line**. But many real relationships are **curved**.

---

### 2) Key Trick: Add Nonlinear Features, Keep the Model Linear in Parameters

Polynomial regression does **not** make the model nonlinear in $\boldsymbol{\theta}$. It makes the **features** nonlinear (powers of $x$), then uses linear regression on those features.

**Example (degree 2):**
* Create features: $x, x^2$
* Fit:

$$\hat{y} = \theta_0 + \theta_1 x + \theta_2 x^2$$

**For degree $d$:**

$$\hat{y} = \theta_0 + \theta_1 x + \theta_2 x^2 + \cdots + \theta_d x^d$$

> 📝 **Key idea:** Polynomial regression is still **"linear regression"** — it is linear with respect to the parameters $\theta_0, \dots, \theta_d$. The nonlinearity is in the **features**, not the model.

---

### 3) Overfitting Risk Increases With Degree

* Higher degree → more flexible curve → fits training data extremely well
* But it may start **fitting noise** → test error gets worse

This is the classic **bias–variance tradeoff**:

| Degree | Bias | Variance | Result |
|---|---|---|---|
| Too low | High | Low | **Underfit** |
| Just right | Balanced | Balanced | ✅ Good generalization |
| Too high | Low | High | **Overfit** |

---

### 4) How to Implement in Scikit-Learn
```python
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

poly_reg = Pipeline([
    ("poly_features", PolynomialFeatures(degree=2, include_bias=False)),
    ("lin_reg", LinearRegression())
])

poly_reg.fit(X_train, y_train)
```

> `PolynomialFeatures` automatically creates all power combinations up to the chosen degree — then `LinearRegression` fits on those expanded features as normal.

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

np.random.seed(42)

# Curved synthetic data
m = 100
X = 6 * np.random.rand(m, 1) - 3          # range ~[-3, 3]
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)  # quadratic + noise

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def fit_and_report(degree):
    model = Pipeline([
        ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
        ("scaler", StandardScaler()),
        ("lin_reg", LinearRegression())
    ])
    model.fit(X_train, y_train)
    train_rmse = rmse(y_train, model.predict(X_train))
    test_rmse  = rmse(y_test,  model.predict(X_test))
    print(f"degree={degree:2d} | train RMSE={train_rmse:.3f} | test RMSE={test_rmse:.3f}")

for d in [1, 2, 10]:
    fit_and_report(d)

degree= 1 | train RMSE=1.777 | test RMSE=1.592
degree= 2 | train RMSE=0.903 | test RMSE=0.797
degree=10 | train RMSE=0.878 | test RMSE=0.813


# 📘 Chapter 4 — Topic 6: Learning Curves

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, find section **"Learning Curves"**

Keywords: `Learning Curves`, `underfitting`, `overfitting`, `bias`, `variance`, `training error`, `validation error`

---

## B) Explanation

### 1) What a Learning Curve Is

A learning curve plots model **error vs. amount of training data**.

We track two curves:
* **Training error** — error on the training subset used to fit the model
* **Validation error** — error on held-out data (or CV folds)

As training size increases:
* Training error usually **goes up** (harder to fit more points perfectly)
* Validation error usually **goes down** (more data helps generalization)

> 📝 **Key idea:** The *gap* between the two curves, and their *absolute levels*, tell you whether you're underfitting or overfitting.

---

### 2) Pattern 1 — Underfitting (High Bias)

**Signs:**
* Training error is **high**
* Validation error is **high**
* They are **close together** (small gap)

**Interpretation:**
> The model is too simple to capture the pattern. Adding more data doesn't help much.

**Typical fixes:**
* Use a more expressive model (e.g., higher polynomial degree)
* Add better features
* Reduce regularization (if applied)

---

### 3) Pattern 2 — Overfitting (High Variance)

**Signs:**
* Training error is **low**
* Validation error is **significantly higher**
* There is a **big gap** between them

**Interpretation:**
> The model is fitting noise — too flexible. More data often reduces the gap but may not be enough alone.

**Typical fixes:**
* Simplify the model (lower degree)
* Add regularization (Ridge / Lasso)
* Get more training data

---

### 4) Full Comparison

| | Underfitting | Overfitting |
|---|---|---|
| Training error | High | Low |
| Validation error | High | High |
| Gap between curves | Small | **Large** |
| Bias | High | Low |
| Variance | Low | High |
| Fix | More complexity | Less complexity / regularization |

---

### 5) The Bias–Variance Tradeoff (Unified View)

$$\text{Total Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Noise}$$

* **Bias** = error from wrong assumptions (model too simple)
* **Variance** = error from sensitivity to training data (model too complex)
* **Irreducible noise** = inherent noise in the data — cannot be reduced

> 📝 **Key idea:** Learning curves are your **diagnostic tool**. Before changing a model, plot learning curves to know *which direction* to move.

In [6]:
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression

np.random.seed(42)

# Same curved synthetic data idea as before
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = (0.5 * X**2 + X + 2 + np.random.randn(m, 1)).ravel()

def report_learning_curve(degree):
    model = Pipeline([
        ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
        ("scaler", StandardScaler()),
        ("lin_reg", LinearRegression())
    ])

    train_sizes, train_scores, val_scores = learning_curve(
        model, X, y,
        cv=5,
        scoring="neg_mean_squared_error",
        train_sizes=np.linspace(0.1, 1.0, 5),
        shuffle=True,
        random_state=42
    )

    train_rmse = np.sqrt(-train_scores)
    val_rmse = np.sqrt(-val_scores)

    print(f"\nDegree = {degree}")
    for s, tr, va in zip(train_sizes, train_rmse.mean(axis=1), val_rmse.mean(axis=1)):
        print(f"  m={int(s):3d} | train RMSE={tr:.3f} | val RMSE={va:.3f}")

report_learning_curve(1)
report_learning_curve(10)


Degree = 1
  m=  8 | train RMSE=1.395 | val RMSE=1.899
  m= 26 | train RMSE=1.764 | val RMSE=1.800
  m= 44 | train RMSE=1.765 | val RMSE=1.756
  m= 62 | train RMSE=1.716 | val RMSE=1.745
  m= 80 | train RMSE=1.738 | val RMSE=1.745

Degree = 10
  m=  8 | train RMSE=0.000 | val RMSE=220.479
  m= 26 | train RMSE=0.748 | val RMSE=1.213
  m= 44 | train RMSE=0.829 | val RMSE=1.065
  m= 62 | train RMSE=0.827 | val RMSE=1.060
  m= 80 | train RMSE=0.842 | val RMSE=1.029


# 📘 Chapter 4 — Topic 7: Regularized Linear Models — Ridge Regression

---

## 🔍 Why Degree 10 Screams "High Variance"

* At $m = 8$, train RMSE = `0.000` → the model **memorizes** those few points (a wiggly curve passes through all of them)
* But val RMSE = `220.479` → generalizes horribly — the curve swings wildly between points, so predictions **explode** for some validation $x$ values
* As $m$ increases (26, 44, 62, 80), validation RMSE drops toward ~1.0 — more data **constrains** the curve, reducing the worst overfitting

> Very low train error + much higher validation error (big gap) ⇒ **high variance / overfitting** ✅

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, find sections **"Regularized Linear Models"** → **"Ridge Regression"**

Keywords: `Regularized Linear Models`, `Ridge`, `L2`, `alpha`, `||θ||^2`

---

## B) Explanation

### 1) Why Regularization?

When a model is too flexible (e.g., high-degree polynomial features), it gets **high variance** (overfits). Regularization combats this by **discouraging large weights**.

---

### 2) Ridge Regression = Linear Regression + L2 Penalty

Ridge modifies the cost function by adding a penalty on the **size of the weights**:

$$J(\boldsymbol{\theta}) = \text{MSE}(\boldsymbol{\theta}) + \alpha \sum_{j=1}^{n}\theta_j^2$$

* $\sum \theta_j^2$ = the **L2 norm squared**
* $\alpha \geq 0$ = controls how strong the penalty is
* The bias term $\theta_0$ is **not penalized**

---

### 3) What $\alpha$ Does (Key Intuition)

| $\alpha$ value | Effect |
|---|---|
| $\alpha = 0$ | Plain Linear Regression — no regularization |
| Small $\alpha$ | Slight shrinkage — close to unregularized |
| Large $\alpha$ | Weights shrink toward 0 → simpler model → lower variance, higher bias |

> 📝 **Key idea:** Ridge does **not** eliminate features — it **shrinks** all weights. It trades a small increase in bias for a potentially large reduction in variance, improving test performance when the unregularized model overfits.

---

### 4) Bias–Variance View of Ridge

$$\text{Total Error} = \underbrace{\text{Bias}^2}_{\uparrow \text{ with } \alpha} + \underbrace{\text{Variance}}_{\downarrow \text{ with } \alpha} + \text{Irreducible Noise}$$

Ridge shifts the tradeoff:
* Unregularized overfit model → high variance dominates
* Adding $\alpha$ → variance drops, bias rises slightly → **net improvement** in test error

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

np.random.seed(42)

m = 100
X = 6 * np.random.rand(m, 1) - 3
y = (0.5 * X**2 + X + 2 + np.random.randn(m, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def eval_model(model, name):
    model.fit(X_train, y_train)
    tr = rmse(y_train, model.predict(X_train))
    te = rmse(y_test, model.predict(X_test))
    print(f"{name:>18s} | train RMSE={tr:.3f} | test RMSE={te:.3f}")

degree = 10

plain = Pipeline([
    ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
    ("scaler", StandardScaler()),
    ("lin", LinearRegression())
])

ridge_1 = Pipeline([
    ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0, random_state=42))
])

ridge_10 = Pipeline([
    ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=10.0, random_state=42))
])

eval_model(plain,   "LinearRegression")
eval_model(ridge_1, "Ridge alpha=1")
eval_model(ridge_10,"Ridge alpha=10")

  LinearRegression | train RMSE=0.878 | test RMSE=0.813
     Ridge alpha=1 | train RMSE=0.893 | test RMSE=0.798
    Ridge alpha=10 | train RMSE=0.968 | test RMSE=0.818


# 📘 Chapter 4 — Topic 8: Lasso Regression (L1 Regularization)

---

## ✅ Ridge α Limits (Quick Reference)

* As $\alpha \to 0$: Ridge → ordinary linear regression (almost no regularization)
* As $\alpha \to \infty$: weights $\theta_1, \theta_2, \dots$ are pushed toward 0 → very simple model, low variance, high bias

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, under **"Regularized Linear Models"** → find **"Lasso Regression"**

Keywords: `Lasso`, `L1`, `Least Absolute Shrinkage`, `feature selection`, `||θ||_1`

---

## B) Explanation

### 1) What Lasso Is

Lasso regression is linear regression with an **L1 penalty** added to the cost:

$$J(\boldsymbol{\theta}) = \text{MSE}(\boldsymbol{\theta}) + \alpha \sum_{j=1}^{n}|\theta_j|$$

---

### 2) Ridge vs Lasso — The Key Behavioral Difference

| | Ridge (L2) | Lasso (L1) |
|---|---|---|
| Penalty term | $\sum \theta_j^2$ | $\sum \|\theta_j\|$ |
| Effect on weights | Shrinks smoothly toward 0 | Can shrink **exactly to 0** |
| Feature selection | ❌ No | ✅ Yes |
| When to prefer | Many small useful features | Sparse model — few features matter |

> 📝 **Key idea:** Lasso performs **automatic feature selection** by driving some coefficients to exactly 0. Ridge never fully eliminates features — it only shrinks them.

---

### 3) What $\alpha$ Does in Lasso

| $\alpha$ value | Effect |
|---|---|
| $\alpha \to 0$ | Close to ordinary linear regression |
| Larger $\alpha$ | More coefficients go to exactly 0 → sparser model → bias ↑, variance ↓ |

---

### 4) Bias–Variance View of Lasso

$$\text{Total Error} = \underbrace{\text{Bias}^2}_{\uparrow \text{ with } \alpha} + \underbrace{\text{Variance}}_{\downarrow \text{ with } \alpha} + \text{Irreducible Noise}$$

> Lasso shifts the same bias–variance tradeoff as Ridge — but its route is **sparsity**: it explicitly removes features rather than just shrinking them.

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error

np.random.seed(42)

m = 100
X = 6 * np.random.rand(m, 1) - 3
y = (0.5 * X**2 + X + 2 + np.random.randn(m, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

degree = 10

plain = Pipeline([
    ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
    ("scaler", StandardScaler()),
    ("lin", LinearRegression())
])

ridge = Pipeline([
    ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0))
])

def eval_lasso(alpha):
    model = Pipeline([
        ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
        ("scaler", StandardScaler()),
        ("lasso", Lasso(alpha=alpha, max_iter=20000))
    ])
    model.fit(X_train, y_train)
    te = rmse(y_test, model.predict(X_test))

    # Count non-zero coefficients (after poly+scaling)
    coefs = model.named_steps["lasso"].coef_
    nnz = np.sum(np.abs(coefs) > 1e-8)
    return te, nnz

plain.fit(X_train, y_train)
ridge.fit(X_train, y_train)

print(f"LinearRegression | test RMSE={rmse(y_test, plain.predict(X_test)):.3f}")
print(f"Ridge(alpha=1)   | test RMSE={rmse(y_test, ridge.predict(X_test)):.3f}")

for a in [0.001, 0.01, 0.1]:
    te, nnz = eval_lasso(a)
    print(f"Lasso(alpha={a}) | test RMSE={te:.3f} | nonzero coefs={nnz}")

LinearRegression | test RMSE=0.813
Ridge(alpha=1)   | test RMSE=0.798
Lasso(alpha=0.001) | test RMSE=0.832 | nonzero coefs=6
Lasso(alpha=0.01) | test RMSE=0.805 | nonzero coefs=5
Lasso(alpha=0.1) | test RMSE=0.797 | nonzero coefs=2


# 📘 Chapter 4 — Topic 9: Elastic Net (L1 + L2 Together)

---

## ✅ Why L1 Creates Sparsity (Quick Reference)

* **L1 penalty** has a "diamond" shape with sharp corners → the optimum often lands on an axis → some $\theta_j = 0$ (sparse)
* **L2 penalty** has a smooth "circle" → shrinks weights but rarely hits exactly 0

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, under **"Regularized Linear Models"** → find **"Elastic Net"**

Keywords: `Elastic Net`, `l1_ratio`, `mix`, `L1`, `L2`

---

## B) Explanation

### 1) What Elastic Net Is

Elastic Net is a **mix of Ridge and Lasso** — it adds both penalties:
* **L1 term** → encourages sparsity (feature selection)
* **L2 term** → encourages small weights and stability

$$J(\boldsymbol{\theta}) = \text{MSE}(\boldsymbol{\theta}) + \alpha\Big(r\sum|\theta_j| + \frac{1-r}{2}\sum \theta_j^2\Big)$$

* $\alpha$ = controls overall regularization strength
* $r$ (`l1_ratio`) = controls the mix:

| $r$ value | Behavior |
|---|---|
| $r = 1$ | Pure Lasso |
| $r = 0$ | Pure Ridge |
| $0 < r < 1$ | Elastic Net (mix of both) |

---

### 2) Why Elastic Net Can Be Better Than Pure Lasso

Elastic Net is especially useful when:
* Features are **strongly correlated** — Lasso may pick one and drop the rest arbitrarily; Elastic Net handles groups of correlated features more stably
* You want **some sparsity but also stability** — L2 prevents erratic weight elimination

---

### 3) Full Regularization Comparison

| Method | Penalty | Sparsity | Stability | Best When |
|---|---|---|---|---|
| Ridge | L2 only | ❌ No | ✅ High | Many small useful features |
| Lasso | L1 only | ✅ Yes | ⚠️ Can be erratic | Few features truly matter |
| **Elastic Net** | L1 + L2 | ✅ Yes | ✅ High | Correlated features / default safe choice |

> 📝 **Key idea:** When in doubt between Ridge and Lasso, **Elastic Net is the safer default** — it inherits the best of both.

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

np.random.seed(42)

m = 100
X = 6 * np.random.rand(m, 1) - 3
y = (0.5 * X**2 + X + 2 + np.random.randn(m, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

degree = 10

def eval_elastic(alpha, l1_ratio):
    model = Pipeline([
        ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
        ("scaler", StandardScaler()),
        ("enet", ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=50000, random_state=42))
    ])
    model.fit(X_train, y_train)
    te = rmse(y_test, model.predict(X_test))
    coefs = model.named_steps["enet"].coef_
    nnz = np.sum(np.abs(coefs) > 1e-8)
    print(f"ElasticNet(alpha={alpha}, l1_ratio={l1_ratio}) | test RMSE={te:.3f} | nonzero coefs={nnz}")

for l1r in [0.2, 0.5, 0.8]:
    eval_elastic(alpha=0.1, l1_ratio=l1r)

ElasticNet(alpha=0.1, l1_ratio=0.2) | test RMSE=0.811 | nonzero coefs=5
ElasticNet(alpha=0.1, l1_ratio=0.5) | test RMSE=0.804 | nonzero coefs=4
ElasticNet(alpha=0.1, l1_ratio=0.8) | test RMSE=0.798 | nonzero coefs=2


# 📘 Chapter 4 — Topic 10: Early Stopping

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, inside or after **"Regularized Linear Models"** → find **"Early Stopping"**

Keywords: `Early Stopping`, `validation`, `overfitting`, `stop`, `best model`

---

## B) Explanation

### 1) What Early Stopping Means

When you train an iterative model (GD / SGD / mini-batch), you typically run for many epochs.

**Early stopping rule:**
* Keep a **validation set**
* Track **validation error** during training
* **Stop training** when validation error stops improving (or starts getting worse)

This prevents the model from entering the **"overfitting zone"**.

---

### 2) Why It Acts Like Regularization

Overfitting often happens because the model keeps **adapting to noise** as training continues.

Early stopping reduces overfitting by **limiting training time** — which effectively limits how complex the fitted solution becomes.

> 📝 **Key idea:** Early stopping = **regularization by stopping before the model overfits**. No penalty term needed — time itself is the constraint.

---

### 3) Practical Implementation Detail (Important)

We don't just "stop at the last epoch" — we **save the best parameters seen on validation** (best epoch), not the final epoch.
```python
from sklearn.base import clone

# Track best model during training
best_val_error = float("inf")
best_model = None

for epoch in range(n_epochs):
    model.partial_fit(X_train, y_train)
    val_error = mean_squared_error(y_val, model.predict(X_val))

    if val_error < best_val_error:
        best_val_error = val_error
        best_model = clone(model)  # save best weights
```

---

### 4) Full Regularization Methods — Chapter 4 Summary

| Method | How It Regularizes | Sparsity |
|---|---|---|
| Ridge | L2 penalty on weights | ❌ |
| Lasso | L1 penalty on weights | ✅ |
| Elastic Net | L1 + L2 penalty | ✅ |
| **Early Stopping** | Limits training time / complexity | ❌ |

> All four methods fight the same enemy: **overfitting**. They just attack it differently.

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

np.random.seed(42)

# Noisy curved data (makes overfitting easier to observe)
m = 200
X = 6 * np.random.rand(m, 1) - 3
y = (0.5 * X**2 + X + 2 + 1.5*np.random.randn(m, 1)).ravel()

# Split: train / val / test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)  # 0.25 of 0.8 = 0.2

degree = 30
poly = PolynomialFeatures(degree=degree, include_bias=False)
scaler = StandardScaler()

X_train_p = scaler.fit_transform(poly.fit_transform(X_train))
X_val_p   = scaler.transform(poly.transform(X_val))
X_test_p  = scaler.transform(poly.transform(X_test))

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

sgd = SGDRegressor(
    random_state=42,
    max_iter=1,      # we control epochs manually
    tol=None,
    learning_rate="invscaling",
    eta0=0.1,
    power_t=0.25,
    penalty="l2",
    alpha=1e-4
)

best_val = float("inf")
best_epoch = None
best_coef = None
best_intercept = None

patience = 20
patience_left = patience

n_epochs = 500

for epoch in range(n_epochs):
    sgd.partial_fit(X_train_p, y_train)

    val_rmse = rmse(y_val, sgd.predict(X_val_p))

    if val_rmse < best_val - 1e-6:
        best_val = val_rmse
        best_epoch = epoch
        best_coef = sgd.coef_.copy()
        best_intercept = sgd.intercept_.copy()
        patience_left = patience
    else:
        patience_left -= 1
        if patience_left == 0:
            break

# Restore best model (this is the key part of early stopping)
sgd.coef_ = best_coef
sgd.intercept_ = best_intercept

test_rmse = rmse(y_test, sgd.predict(X_test_p))

print(f"degree={degree}")
print(f"stopped_epoch={epoch}, best_epoch={best_epoch}")
print(f"best_val_RMSE={best_val:.3f}")
print(f"test_RMSE={test_rmse:.3f}")

degree=30
stopped_epoch=20, best_epoch=0
best_val_RMSE=3128.278
test_RMSE=4875.840


## Why We Keep the Best Epoch (Not the Final Epoch)

When you train iteratively (SGD / mini-batch / GD), two things happen over time:

1. **Training error** usually keeps improving → the model fits the training set better and better
2. **Validation error** improves at first, then often **gets worse** once the model starts overfitting (fitting noise / idiosyncrasies in the training set)

> The best generalization happens at the epoch where **validation error is minimal** — not the last epoch.

---

## The Key Point

Early stopping works like this:

* Monitor validation RMSE each epoch
* When it stops improving, don't stop instantly (validation can be noisy) → **wait a bit** (patience)
* That means the final epoch is often **after** the best epoch
* Therefore: **save and restore weights from the best validation epoch**, not the last epoch you happened to run

> 📝 **In one sentence:** We keep the best validation model because the last trained weights may already be **"past the sweet spot"** — starting to overfit or drift.

---

## Why `best_epoch = 0` and Huge RMSE Happens

If your output shows `best_epoch=0` and validation/test RMSE in the thousands, this usually means:

* Training is **numerically unstable / diverging** — steps are too aggressive for this feature space
* With **degree=30 polynomial features**, SGD becomes extremely sensitive
* The very first epoch already overshoots, so epoch 0 (before any update) looks like the "best"

**Fixes:**
* Reduce polynomial degree (degree=30 is extreme)
* Lower the learning rate (`eta0`)
* Add feature scaling (`StandardScaler`) before fitting — SGD is very sensitive to feature scale

> 📝 **Key insight:** Early stopping can only save you if training is stable enough to improve in the first place. Diverging training means the learning rate or feature scale needs fixing first.

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

np.random.seed(42)

m = 200
X = 6 * np.random.rand(m, 1) - 3
y = (0.5 * X**2 + X + 2 + 1.5*np.random.randn(m, 1)).ravel()

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

degree = 20  # lowered from 30
poly = PolynomialFeatures(degree=degree, include_bias=False)
scaler = StandardScaler()

X_train_p = scaler.fit_transform(poly.fit_transform(X_train))
X_val_p   = scaler.transform(poly.transform(X_val))
X_test_p  = scaler.transform(poly.transform(X_test))

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

sgd = SGDRegressor(
    random_state=42,
    max_iter=1,
    tol=None,
    learning_rate="invscaling",
    eta0=0.01,      # lowered from 0.1
    power_t=0.5,
    penalty="l2",
    alpha=1e-3,     # stronger regularization
    average=True    # stabilizes SGD a lot
)

best_val = float("inf")
best_epoch = None
best_coef = None
best_intercept = None

patience = 20
patience_left = patience

n_epochs = 500

for epoch in range(n_epochs):
    sgd.partial_fit(X_train_p, y_train)
    val_rmse = rmse(y_val, sgd.predict(X_val_p))

    if val_rmse < best_val - 1e-6:
        best_val = val_rmse
        best_epoch = epoch
        best_coef = sgd.coef_.copy()
        best_intercept = sgd.intercept_.copy()
        patience_left = patience
    else:
        patience_left -= 1
        if patience_left == 0:
            break

sgd.coef_ = best_coef
sgd.intercept_ = best_intercept

test_rmse = rmse(y_test, sgd.predict(X_test_p))

print(f"degree={degree}")
print(f"stopped_epoch={epoch}, best_epoch={best_epoch}")
print(f"best_val_RMSE={best_val:.3f}")
print(f"test_RMSE={test_rmse:.3f}")

degree=20
stopped_epoch=499, best_epoch=499
best_val_RMSE=1.508
test_RMSE=1.652


# 📘 Chapter 4 — Topic 11: Logistic Regression (Binary Classification)

---

## ✅ Early Stopping Patience — Small Fix

Patience is **not** about escaping local minima (for linear models the objective is convex — no local minima issue).

It's about **noise in the validation metric**: validation RMSE can wiggle up/down epoch to epoch (especially with SGD / shuffling). We wait `patience` epochs to confirm improvement really stopped — not just a random blip.

> `stopped_epoch=499, best_epoch=499` → validation kept improving the whole run, so early stopping never triggered. ✅

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, find section **"Logistic Regression"** and subsections:

| Subsection | Keywords |
|---|---|
| Intro + probability model | `Logistic Regression`, `sigmoid`, `logistic function` |
| Decision boundary | `decision boundary`, `predict_proba`, `0.5` |
| Cost function | `log loss`, `cross entropy`, `cost function` |
| Multiclass extension | `Softmax Regression`, `multinomial` |

---

## B) Explanation

### 1) What Logistic Regression Is

Despite the name, logistic regression is a **classifier**.

It starts like linear regression — compute a linear score:

$$z = \boldsymbol{\theta}^\top \mathbf{x}$$

Then converts that score into a **probability** using the sigmoid:

$$\hat{p} = \sigma(z) = \frac{1}{1 + e^{-z}}$$

**Interpretation:**

$$\hat{p} = P(y = 1 \mid \mathbf{x})$$

> 📝 **Key idea:** Logistic regression doesn't predict a value — it predicts a **probability**, then thresholds it into a class.

---

### 2) Prediction Rule (Thresholding)

$$\hat{y} = \begin{cases} 1 & \text{if } \hat{p} \geq 0.5 \\ 0 & \text{if } \hat{p} < 0.5 \end{cases}$$

Because $\sigma(z) \geq 0.5 \iff z \geq 0$, the decision boundary is:

$$\boldsymbol{\theta}^\top \mathbf{x} = 0$$

> The boundary is **linear in feature space** — a line in 2D, a plane in 3D, a hyperplane in higher dims.

---

### 3) How It's Trained (Log Loss / Cross-Entropy)

Instead of MSE, logistic regression minimizes **log loss (cross-entropy)**:

* If $y = 1$: loss $= -\log(\hat{p})$
* If $y = 0$: loss $= -\log(1 - \hat{p})$

**Intuition:**
* If $y=1$ and $\hat{p} \to 1$ → loss $\to 0$ ✅
* If $y=1$ and $\hat{p} \to 0$ → loss $\to \infty$ ❌ (heavily penalized)

Over all training data, we minimize the **average log loss** (often + regularization).

---

### 4) Regularization (Tie-in to Ridge/Lasso)

In scikit-learn, logistic regression is **regularized by default** (commonly L2).

| Parameter | Effect |
|---|---|
| `C` (sklearn) | $C = \frac{1}{\alpha}$ — **smaller C = stronger regularization** |
| `penalty='l2'` | Ridge-style — shrinks weights |
| `penalty='l1'` | Lasso-style — can zero out weights |

> 📝 **Key idea:** The same bias–variance tradeoff from Ridge/Lasso applies here. Regularization prevents logistic regression from overfitting when features are many or correlated.

In [12]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

iris = load_iris()
X = iris.data  # 4 features
y = (iris.target == 2).astype(int)  # 1 if Virginica else 0

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = Pipeline([
    ("scaler", StandardScaler()),
    ("log_reg", LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42))
])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
proba = model.predict_proba(X_test)[:5]  # first 5 probability rows

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("First 5 predicted probabilities [P(class0), P(class1)]:\n", np.round(proba, 4))

# optional: show learned parameters
lr = model.named_steps["log_reg"]
print("Intercept:", np.round(lr.intercept_, 4))
print("Coefficients:", np.round(lr.coef_, 4))

Accuracy: 1.0
Confusion matrix:
 [[20  0]
 [ 0 10]]
First 5 predicted probabilities [P(class0), P(class1)]:
 [[0.791  0.209 ]
 [0.4184 0.5816]
 [0.7159 0.2841]
 [0.9461 0.0539]
 [1.     0.    ]]
Intercept: [-3.5173]
Coefficients: [[ 0.2788 -0.4579  2.1206  2.8669]]


# 📘 Chapter 4 — Topic 12: Softmax Regression (Multiclass Logistic Regression)

---

## A) Where to Read (Pinpoint)

From `4_Training Models.pdf`, find **"Softmax Regression"** (near end of Logistic Regression section)

Keywords: `Softmax`, `multinomial`, `cross entropy`, `log loss`, `K classes`

---

## B) Explanation

### 1) Why We Need Softmax

Binary logistic regression models $P(y=1 \mid \mathbf{x})$.

But if $y$ can be $0, 1, 2, \dots, K-1$, we want a **probability for each class** — and they must sum to 1.

---

### 2) Model: One Linear Score Per Class

For each class $k$, compute a score (logit):

$$s_k(\mathbf{x}) = \boldsymbol{\theta}_k^\top \mathbf{x}$$

Collect into a vector of scores $\mathbf{s}(\mathbf{x})$.

> This is the same idea as OvR from Chapter 3 — one scorer per class — but now trained **jointly** instead of independently.

---

### 3) Convert Scores to Probabilities With Softmax

$$\hat{p}_k = P(y=k \mid \mathbf{x}) = \frac{e^{s_k(\mathbf{x})}}{\sum_{j=0}^{K-1} e^{s_j(\mathbf{x})}}$$

**Properties:**
* All $\hat{p}_k \geq 0$
* $\sum_k \hat{p}_k = 1$ (valid probability distribution)
* The largest score gets the largest probability

**Prediction rule:**

$$\hat{y} = \arg\max_k \hat{p}_k = \arg\max_k s_k(\mathbf{x})$$

> Since Softmax is monotonic in the scores, argmax of probabilities = argmax of raw scores.

---

### 4) Training Objective: Cross-Entropy (Multiclass Log Loss)

For one training example, the loss is:

$$-\log(\hat{p}_{\text{true class}})$$

* If $\hat{p}_{\text{true class}} \to 1$ → loss $\to 0$ ✅
* If $\hat{p}_{\text{true class}} \to 0$ → loss $\to \infty$ ❌ (heavily penalized)

Average over the dataset (+ regularization).

---

### 5) Full Picture — Logistic vs Softmax

| | Logistic Regression | Softmax Regression |
|---|---|---|
| Classes | 2 (binary) | $K$ (multiclass) |
| Scores | 1 score → sigmoid | $K$ scores → softmax |
| Output | $P(y=1 \mid \mathbf{x})$ | $P(y=k \mid \mathbf{x})$ for all $k$ |
| Loss | Binary cross-entropy | Multiclass cross-entropy |
| Decision boundary | Linear hyperplane | $K$ linear hyperplanes |

> 📝 **Key idea:** Softmax is the natural generalization of logistic regression to $K$ classes. Same linear foundation, same cross-entropy loss — just extended to output a full probability distribution over classes.

In [13]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

iris = load_iris()
X = iris.data
y = iris.target  # 3 classes: 0,1,2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

softmax_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("log_reg", LogisticRegression(
        multi_class="multinomial",
        solver="lbfgs",
        C=10.0,          # weaker regularization than default
        max_iter=2000,
        random_state=42
    ))
])

softmax_clf.fit(X_train, y_train)

y_pred = softmax_clf.predict(X_test)
proba = softmax_clf.predict_proba(X_test)[:5]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("First 5 predicted probability vectors:\n", np.round(proba, 4))

lr = softmax_clf.named_steps["log_reg"]
print("Coef shape:", lr.coef_.shape)      # (K, n_features)
print("Intercept shape:", lr.intercept_.shape)  # (K,)

Accuracy: 1.0
Confusion matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]
First 5 predicted probability vectors:
 [[9.982e-01 1.800e-03 0.000e+00]
 [1.000e-04 3.084e-01 6.915e-01]
 [8.010e-02 9.197e-01 2.000e-04]
 [3.760e-02 9.621e-01 2.000e-04]
 [9.993e-01 7.000e-04 0.000e+00]]
Coef shape: (3, 4)
Intercept shape: (3,)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


# 📘 Chapter 4 — Softmax: Invariance to Constant Shifts

---

## Why Adding a Constant Doesn't Change Softmax (2-Line Proof)

Let the original scores be $s_0, s_1, \dots, s_{K-1}$. Softmax is:

$$\hat{p}_k = \frac{e^{s_k}}{\sum_j e^{s_j}}$$

If we add a constant $c$ to every score:

$$\hat{p}'_k = \frac{e^{s_k+c}}{\sum_j e^{s_j+c}} = \frac{e^c e^{s_k}}{e^c \sum_j e^{s_j}} = \frac{e^{s_k}}{\sum_j e^{s_j}} = \hat{p}_k$$

> The $e^c$ cancels exactly. Probabilities are **identical**. ✅

---

## Practical Note — Numerically Stable Softmax

Because of this invariance, implementations compute:

$$\text{softmax}(\mathbf{s}) = \text{softmax}(\mathbf{s} - \max(\mathbf{s}))$$

This **prevents overflow** (huge exponentials from large scores) while keeping all probabilities identical.

---

## About the Scikit-Learn Warning

The `multi_class=` parameter is being deprecated — Softmax-style multinomial handling will become the default.

Silence it by removing `multi_class="multinomial"`:
```python
LogisticRegression(
    solver="lbfgs",
    C=10.0,
    max_iter=2000,
    random_state=42
)
```

> 📝 **Key idea:** The invariance proof is not just a math curiosity — it's why numerically stable Softmax implementations subtract the max score before exponentiating. Same math, no overflow.

# 📘 Chapter 4 "Training Models" — Lock-In Notes

---

## A) Where to Read (Pinpoint References in Order)

| Section | Keywords |
|---|---|
| Linear Regression | `Linear Regression`, `MSE`, `cost function`, `theta` |
| The Normal Equation | `Normal Equation`, `X^T X`, `inverse`, `pseudoinverse` |
| Gradient Descent (all variants) | `Gradient Descent`, `learning rate`, `eta`, `Stochastic`, `Mini-batch`, `schedule` |
| Polynomial Regression | `Polynomial Regression`, `PolynomialFeatures`, `degree` |
| Learning Curves | `Learning Curves`, `bias`, `variance`, `underfitting`, `overfitting` |
| Regularized Linear Models | `Regularized Linear Models`, `Ridge`, `Lasso`, `Elastic Net`, `alpha` |
| Early Stopping | `Early Stopping`, `validation`, `patience` |
| Logistic Regression | `Logistic Regression`, `sigmoid`, `log loss`, `cross entropy`, `decision boundary` |
| Softmax Regression | `Softmax`, `multinomial`, `cross entropy` |

---

## B) Chapter 4 Lock-In Notes

### 1) Linear Regression (Core Idea)

**Model:**

1 feature: $\hat{y} = \theta_0 + \theta_1 x$

$n$ features — add bias feature $x_0 = 1$, so $\mathbf{x} = [1, x_1, \dots, x_n]$ and:

$$\hat{y} = \mathbf{x}^\top \boldsymbol{\theta}$$

**Objective (MSE cost):**

$$J(\boldsymbol{\theta}) = \frac{1}{m}\sum_{i=1}^{m}(\hat{y}^{(i)} - y^{(i)})^2$$

> Training = choose $\boldsymbol{\theta}$ that minimizes MSE.

**Matrix view:** Stack all instances into matrix $\mathbf{X}$ (with bias column of 1s), targets into vector $\mathbf{y}$, predictions: $\hat{\mathbf{y}} = \mathbf{X}\boldsymbol{\theta}$

---

### 2) Normal Equation (Closed-Form Solution)

$$\hat{\boldsymbol{\theta}} = (\mathbf{X}^\top \mathbf{X})^{-1}\mathbf{X}^\top \mathbf{y}$$

Jump straight to the MSE minimum — no iterations.

* **When it's great:** small/medium number of features
* **When it's not:** huge feature counts (matrix operations become heavy)
* **Practical note:** implementations use stable methods (pseudo-inverse / SVD) rather than a literal inverse

---

### 3) Gradient Descent Family (Iterative Optimization)

**General update rule:**

$$\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \eta \nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$$

$\eta$ = learning rate: too small → slow, too large → overshoot / divergence

| Variant | Instances per step | Path | Best for |
|---|---|---|---|
| Batch GD | All $m$ | Smooth, expensive | Small datasets |
| SGD | 1 random | Noisy, fast | Large datasets |
| Mini-batch GD | $b$ instances | Middle ground | **Default in practice** |

**Batch GD gradient (for linear regression):**

$$\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta}) = \frac{2}{m}X_b^\top(X_b\boldsymbol{\theta} - \mathbf{y})$$

**Learning schedule** (important with SGD/mini-batch): decrease $\eta$ over time — big steps early → fast progress, smaller steps later → settle near minimum.

---

### 4) Polynomial Regression (Curves Using Linear Models)

**Trick:** make features nonlinear, keep the model linear in parameters.

Degree $d$:

$$\hat{y} = \theta_0 + \theta_1 x + \theta_2 x^2 + \cdots + \theta_d x^d$$

> 📝 Polynomial regression is still "linear" because it's linear in the $\theta$'s.

**Risk:** higher degree → higher flexibility → higher overfitting risk.

---

### 5) Learning Curves (Diagnose Underfit vs Overfit)

Plot training error and validation error vs training set size.

| Pattern | Train error | Val error | Gap | Fix |
|---|---|---|---|---|
| **Underfitting** (high bias) | High | High | Small | Richer model, less regularization |
| **Overfitting** (high variance) | Low | Higher | **Big** | Simplify, regularize, more data |

---

### 6) Regularized Linear Models (Control Overfitting by Shrinking Weights)

**Ridge (L2):**

$$J(\boldsymbol{\theta}) = \text{MSE}(\boldsymbol{\theta}) + \alpha \sum_{j=1}^{n}\theta_j^2$$

Larger $\alpha$ → weights shrink more → variance ↓, bias ↑

**Lasso (L1):**

$$J(\boldsymbol{\theta}) = \text{MSE}(\boldsymbol{\theta}) + \alpha \sum_{j=1}^{n}|\theta_j|$$

Can drive some $\theta_j$ to exactly 0 → **feature selection** (sparse model)

**Elastic Net (L1 + L2):**

$$J(\boldsymbol{\theta}) = \text{MSE}(\boldsymbol{\theta}) + \alpha\Big(r\sum|\theta_j| + \frac{1-r}{2}\sum\theta_j^2\Big)$$

L1 helps sparsity, L2 helps stability. Use when features are correlated and you want "some sparsity, but not unstable."

---

### 7) Early Stopping (Regularization by Stopping at Best Validation)

* Track validation error during training
* Stop when validation error stops improving (with **patience**)
* Keep the **best epoch weights** (lowest val error), not the last weights

> **Why patience exists:** validation error can bounce slightly (noise), so you wait before deciding improvement is truly over.

---

### 8) Logistic Regression (Binary Classification)

**Probability model:**

$$\hat{p} = \sigma(z) = \frac{1}{1+e^{-z}} = P(y=1 \mid \mathbf{x}), \quad z = \boldsymbol{\theta}^\top \mathbf{x}$$

**Decision rule:** predict 1 if $\hat{p} \geq 0.5$, else 0.
Since $\sigma(z) = 0.5 \iff z = 0$, decision boundary: $\boldsymbol{\theta}^\top \mathbf{x} = 0$ (linear in feature space)

**Training loss (log loss / cross-entropy):**
* if $y=1$: $-\log(\hat{p})$
* if $y=0$: $-\log(1-\hat{p})$

Minimize average loss (+ regularization). `LogisticRegression` is regularized by default in scikit-learn.

---

### 9) Softmax Regression (Multiclass Logistic Regression)

**Scores (one per class):** $s_k(\mathbf{x}) = \boldsymbol{\theta}_k^\top \mathbf{x}$

**Softmax probabilities:**

$$\hat{p}_k = \frac{e^{s_k(\mathbf{x})}}{\sum_j e^{s_j(\mathbf{x})}}$$

**Predict class:** $\arg\max_k \hat{p}_k$ (equivalently $\arg\max_k s_k$)

**Cross-entropy loss:** $-\log(\hat{p}_{\text{true class}})$ for each example

**Numerical stability trick:** softmax doesn't change if you add the same constant to all scores, so compute:

$$\text{softmax}(\mathbf{s}) = \text{softmax}(\mathbf{s} - \max(\mathbf{s}))$$

(prevents overflow from huge exponentials)

---

## C) Code Patterns to Own (Templates)

### 1) Linear Regression — Normal Equation Style
```python
X_b = np.c_[np.ones((m, 1)), X]           # add bias column
theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
```

### 2) Book-Style Pipelines (Safest Default)
```python
Pipeline([
    ("poly",    PolynomialFeatures(degree=d, include_bias=False)),
    ("scaler",  StandardScaler()),
    ("model",   Ridge() / Lasso() / ElasticNet() / LogisticRegression())
])
```

> **Rule of thumb:** if you use GD/SGD or regularization, **scale features**.

### 3) Early Stopping (Mental Model)
```python
# Train epoch by epoch
# Track validation RMSE (or log loss)
# Save best weights
# Stop after patience runs out
# Restore best weights
```

# 📘 Chapter 4 — Exercises: Lock-In Q&A

---

**Q1: What Linear Regression training algorithm can you use if you have a training set with millions of features?**

Use an iterative method such as **Stochastic Gradient Descent** or **Mini-batch Gradient Descent**. The Normal Equation is impractical at that scale — inverting $(\mathbf{X}^\top \mathbf{X})$ becomes computationally prohibitive with millions of features.

---

**Q2: Suppose the features in your training set have very different scales. What algorithms might suffer, and what can you do?**

* **Gradient Descent** (Batch/SGD/Mini-batch) becomes very slow — the cost function becomes poorly conditioned (long, narrow valleys)
* **Regularized models** (Ridge/Lasso/Elastic Net/Logistic Regression) behave poorly — the penalty is affected by scale, so some features dominate unfairly

> ✅ Fix: Scale features with `StandardScaler`, ideally inside a `Pipeline`.

---

**Q3: Can Gradient Descent get stuck in a local minimum when training a Logistic Regression model?**

**No.** Logistic regression's log-loss objective is **convex** — it has a single global optimum, so there are no problematic local minima.

---

**Q4: Do all Gradient Descent algorithms lead to the same model if you let them run long enough?**

For convex problems (linear/logistic regression):
* **Batch GD** converges to the global optimum (with a suitable learning rate)
* **SGD/Mini-batch** may keep oscillating around the optimum if the learning rate doesn't decay

> ✅ Fix: Use a **learning schedule** (decreasing $\eta$) to make SGD/mini-batch truly converge.

---

**Q5: You use Batch GD and notice the validation error consistently goes up at every epoch. What's happening? How do you fix it?**

Most likely **overfitting** — training error keeps improving but generalization worsens.

Fixes: early stopping, regularization (Ridge/Lasso/Elastic Net), reduce model complexity, get more data.

---

**Q6: Is it a good idea to stop Mini-batch GD immediately when the validation error goes up?**

**No.** Mini-batch/SGD validation error is **noisy** and can rise temporarily due to randomness. Use **patience** — wait several epochs before deciding — and always keep the best validation weights.

---

**Q7: Which GD algorithm reaches the vicinity of the optimal solution fastest? Which actually converges? How can you make the others converge?**

* **Fastest to get near optimum:** SGD (and often mini-batch in practice)
* **Most likely to converge smoothly:** Batch GD
* **Make SGD/mini-batch converge:** use a learning schedule (decrease $\eta$ over time)

---

**Q8: You're using Polynomial Regression and notice a large gap between training and validation error. What's happening? Three ways to fix it?**

This is **overfitting (high variance)**.

Three fixes:
1. **Lower the polynomial degree** (simplify the model)
2. **Add regularization** (Ridge / Lasso / Elastic Net)
3. **Get more training data** (often reduces the gap)

*(Also acceptable: early stopping, feature selection.)*

---

**Q9: You use Ridge Regression and notice training error and validation error are almost equal and fairly high. High bias or high variance? Increase α or reduce it?**

That's **high bias (underfitting)** — the model is too constrained.

> ✅ Fix: **Reduce $\alpha$** (less regularization) and/or use a more expressive model or better features.

---

**Q10: Why use Ridge instead of plain Linear Regression? Lasso instead of Ridge? Elastic Net instead of Lasso?**

| Choice | Why |
|---|---|
| **Ridge over plain LR** | Reduces overfitting/variance by shrinking weights; helps with multicollinearity |
| **Lasso over Ridge** | Can set some coefficients **exactly to 0** → automatic feature selection (sparse model) |
| **Elastic Net over Lasso** | More stable when features are correlated; can keep groups of related features that Lasso might drop arbitrarily |

---

**Q11: You want to classify pictures as outdoor/indoor AND daytime/nighttime. Two Logistic Regression classifiers or one Softmax Regression?**

Use **two separate Logistic Regression classifiers** — one per binary label.

Softmax is for a **single multiclass label with mutually exclusive classes**. Here you have two independent binary decisions, so two binary classifiers is the correct framing.

---

**Q12: Implement Batch Gradient Descent with early stopping for Softmax Regression (without Scikit-Learn).**

> See code cell below — pure NumPy implementation of Softmax Regression trained with Batch GD + early stopping (patience, best validation weights).

In [14]:
import numpy as np

def softmax(logits: np.ndarray) -> np.ndarray:
    z = logits - logits.max(axis=1, keepdims=True)  # stability
    exp = np.exp(z)
    return exp / exp.sum(axis=1, keepdims=True)

def one_hot(y: np.ndarray, K: int) -> np.ndarray:
    Y = np.zeros((y.shape[0], K))
    Y[np.arange(y.shape[0]), y] = 1.0
    return Y

def standardize_fit(X: np.ndarray):
    mu = X.mean(axis=0, keepdims=True)
    sigma = X.std(axis=0, keepdims=True) + 1e-12
    return mu, sigma

def standardize_transform(X: np.ndarray, mu: np.ndarray, sigma: np.ndarray):
    return (X - mu) / sigma

def add_bias(X: np.ndarray) -> np.ndarray:
    return np.c_[np.ones((X.shape[0], 1)), X]

def cross_entropy_loss(Xb: np.ndarray, y: np.ndarray, Theta: np.ndarray, l2: float) -> float:
    probs = softmax(Xb @ Theta)
    m = Xb.shape[0]
    loss = -np.log(probs[np.arange(m), y] + 1e-15).mean()
    loss += 0.5 * l2 * np.sum(Theta[1:, :] ** 2)  # don't penalize bias row
    return loss

def train_softmax_bgdearly(
    X_train: np.ndarray, y_train: np.ndarray,
    X_val: np.ndarray, y_val: np.ndarray,
    lr: float = 0.1,
    n_epochs: int = 2000,
    l2: float = 1e-3,
    patience: int = 20,
    tol: float = 1e-6,
    seed: int = 42
):
    rng = np.random.default_rng(seed)
    K = int(max(y_train.max(), y_val.max()) + 1)

    # Standardize features
    mu, sigma = standardize_fit(X_train)
    Xtr = standardize_transform(X_train, mu, sigma)
    Xva = standardize_transform(X_val, mu, sigma)

    Xtr_b = add_bias(Xtr)
    Xva_b = add_bias(Xva)

    n = Xtr_b.shape[1]
    Theta = rng.normal(0, 0.01, size=(n, K))

    best_Theta = Theta.copy()
    best_val = float("inf")
    patience_left = patience

    Ytr = one_hot(y_train, K)
    m = Xtr_b.shape[0]

    for epoch in range(n_epochs):
        probs = softmax(Xtr_b @ Theta)                 # (m, K)
        grad = (Xtr_b.T @ (probs - Ytr)) / m           # (n, K)
        grad[1:, :] += l2 * Theta[1:, :]               # L2 (skip bias)

        Theta -= lr * grad

        val_loss = cross_entropy_loss(Xva_b, y_val, Theta, l2)

        if val_loss < best_val - tol:
            best_val = val_loss
            best_Theta = Theta.copy()
            patience_left = patience
        else:
            patience_left -= 1
            if patience_left == 0:
                break

    return {"Theta": best_Theta, "mu": mu, "sigma": sigma, "epochs_ran": epoch + 1, "best_val_loss": best_val}